In [1]:
!pip install --upgrade pip
!pip install trulens_eval langchain
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.3/751.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [1]:
from langchain.chains import RetrievalQA
# from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import Chroma, Vectara
from langchain.embeddings import OpenAIEmbeddings

from langchain.chat_models.openai import ChatOpenAI
from langchain.document_loaders.unstructured import UnstructuredFileLoader


from langchain import hub
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
import os

customer_id = '189024573'
corpus_id = '2'
api_key = 'zwt_C0RJPZuSyfbGvNCpYBcC1QVBEa_4i2efJ_6lNQ'
openai_key = 'sk-proj-WuCaUIQvwzPVdykSI1y5T3BlbkFJv3RvXynRREtCK5KgopkG'
os.environ["VECTARA_CUSTOMER_ID"] = customer_id
os.environ["VECTARA_CORPUS_ID"] = corpus_id
os.environ["VECTARA_API_KEY"] = api_key
os.environ["OPENAI_API_KEY"] = openai_key

In [3]:
# Imports main tools:
from trulens_eval import TruChain, Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()
tru.reset_database()

# Imports from LangChain to build app
import bs4
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [4]:
vectara = Vectara()
summary_config = {
    "is_enabled": True, "max_results": 10,
    "response_lang": "en",
    "prompt_name": "Generate MCQ"
}
retriever = vectara.as_retriever(
    search_kwargs={"k": 100, "summary_config": summary_config}
)

In [5]:
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
### Contextualize question ###
contextualize_q_system_prompt = """
Given a chat history and the user topic \
which might reference context in the chat history, formulate a MCQ question. \
which can be understood without the chat history. Do NOT answer the question, \
Recreate the question based on the retrieved documents with out giving same question as it is.
"""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)



### Answer question ###
qa_system_prompt = """
You are an assistant for creating MCQ questions based on the context for user. \
Don not display exactly same question, shufle the options and recreate the question. \
Use the following pieces of retrieved context to formulate the MCQ question. \
Do not repeat the question. \
Display one MCQ question and options only at a time. \
Question in one line and options in separate line and wait for the user Answer. \
If user answer correct say your answer is correct, and ask would you like to try another question from same topic.\
If user answer is wrong say your answer is wrong and display the correct answer with explanation, and ask would you like to try another question from same topic.\
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [7]:
# docs = history_aware_retriever.invoke({"input":"animals","chat_history":[]})
# question_answer_chain.invoke({"input":"animals", "context":docs,"chat_history":[]})

In [8]:
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str, chat_history: list) -> list:
        """
        Retrieve relevant text from vector store.
        """
        return history_aware_retriever.invoke({"input": query, "chat_history":chat_history})

    #def generate_completion(self, query: str, context_str: list) -> str:

    @instrument
    def query(self, user_input: str, chat_history: list) -> str:
        return question_answer_chain.invoke( {"input": user_input,"context":self.retrieve(user_input, chat_history),"chat_history":chat_history})
        # return completion


rag = RAG_from_scratch()

In [9]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI

import numpy as np

provider = OpenAI()

grounded = Groundedness(groundedness_provider=provider)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input context will be set to __record__.app.retrieve.rets.collect() .


In [10]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG v1',
    feedbacks = [f_answer_relevance])

In [11]:
with tru_rag as recording:
    chat_history = []
    while True:
        user_input = input("Enter the response: ")
        if user_input == "exit":
            print("bye!")
            break
        chat_history.append(user_input)
        result = rag.query(user_input, chat_history)
        chat_history.append(result)
        print(result)

Enter the response: animal
Your answer is correct. Would you like to try another question from the same topic?
Enter the response: yes
What is the only class of animals that has hair?

A: birds
B: reptiles
C: amphibians
D: mammals
Enter the response: D
Your answer is correct. Would you like to try another question from the same topic?
Enter the response: yes
Mammals are endothermic vertebrates that have four limbs and produce what type of eggs?

A: epithelial
B: gymnoic
C: umbilical
D: amniotic
Enter the response: D
Your answer is correct. Would you like to try another question from the same topic?
Enter the response: organism
It seems like there might be a misunderstanding. If you're looking to continue with questions or need information on a specific topic related to organisms, please specify, and I can assist you accordingly!
Enter the response: give questions on organisms in biology
What type of organism uses sunlight to produce its own food through the process of photosynthesis?



In [12]:
tru.get_leaderboard(app_ids=["RAG v1"])

,Answer Relevance,latency,total_cost
app_id,,,
RAG v1,0.885,10.1,0.1362


In [13]:
tru.run_dashboard()

Starting dashboard ...
npx: installed 22 in 3.843s

Go to this url and submit the ip given here. your url is: https://eight-turtles-say.loca.lt

  Submit this IP Address: 34.81.51.91



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>